In [11]:
import timm 
import torch
from PIL import Image
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
from torchvision.models import ResNet50_Weights

In [12]:
model = timm.create_model('resnet50', pretrained=True)
model = model.eval()

In [13]:
image = Image.open('./src/spider.jpg').convert('RGB')
transform = create_transform(**resolve_data_config(model.pretrained_cfg, model=model))
batch = transform(image).unsqueeze(0)

In [15]:
prediction = model(batch).squeeze(0).softmax(0)

class_id = prediction.argmax().item()
score = prediction[class_id].item()

weights = ResNet50_Weights.DEFAULT
category_name = weights.meta["categories"][class_id]

print(f"{category_name}: {100 * score:.1f}%")

black widow: 100.0%


In [16]:
module = torch.jit.trace(model, batch)
module.save("traced.pt")

In [17]:
import segmentation_models_pytorch as smp

In [19]:
smpModel = smp.Unet(
    encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1000,                      # model output channels (number of classes in your dataset)
)